In [49]:
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess
import librosa
import numpy as np

from IPython.display import Audio as IPythonAudio

In [50]:

import numpy as np
import io

# Parameters for recording
duration = 5  # seconds
sample_rate = 22050  # Hz, the default for librosa or your model



In [51]:
audio_path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Algorithm\audio_text_llm\SenseVoiceSmall\audio_sample\听故事学中文 Learn Chinese with 12 Stories - The Easiest Way to Improve Chinese.mp3'
speech, sample_rate = librosa.load(audio_path, sr = None)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16808\550681422.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sample_rate = librosa.load(audio_path, sr = None)
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [52]:
# If the sample rate is not 16,000 Hz, resample the audio
target_sample_rate = 16000
if sample_rate != target_sample_rate:
    speech = librosa.resample(speech, orig_sr=sample_rate, target_sr=target_sample_rate)
    sample_rate = target_sample_rate  # Update the sample rate

speech = speech[:int(900000)]

In [71]:
import numpy as np
import librosa
from scipy.signal import butter, lfilter

# Function to create a band-pass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Function to apply the band-pass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Set your low and high cut-off frequencies (in Hz)
lowcut = 1000.0
highcut = 3400.0

# Apply the band-pass filter
filtered_speech = bandpass_filter(speech, lowcut, highcut, target_sample_rate)


In [72]:
# Print details
print("Sample Rate:", sample_rate)
print("Audio Array:", filtered_speech)
print("Audio Array datatype is Array: ", isinstance(speech,np.ndarray) )

Sample Rate: 16000
Audio Array: [8.80059615e-13 5.30707346e-12 1.38455081e-11 ... 6.20456824e-05
 3.45207431e-05 1.03955002e-05]
Audio Array datatype is Array:  True


In [73]:
chunk_size = [0, 10, 5]  # [0, 10, 5] 600ms, [0, 8, 4] 480ms
encoder_chunk_look_back = 4  # number of chunks to lookback for encoder self-attention
decoder_chunk_look_back = 1  # number of encoder chunks to lookback for decoder cross-attention

model = AutoModel(model="paraformer-zh-streaming", model_revision="v2.0.4")

New version available: 1.1.9. Your current version is 1.1.2.
Please use the command "pip install -U funasr" to upgrade.


2024-10-08 14:20:43,582 - modelscope - INFO - Use user-specified model revision: v2.0.4


In [74]:
chunk_stride = chunk_size[1] * 960  # 600ms
transcribed_text = []

cache = {}
total_chunk_num = int(len((filtered_speech) - 1) / chunk_stride + 1)
for i in range(total_chunk_num):
    speech_chunk = filtered_speech[i * chunk_stride:(i + 1) * chunk_stride]
    is_final = i == total_chunk_num - 1
    res = model.generate(input=speech_chunk, cache=cache, is_final=is_final, chunk_size=chunk_size,
                         encoder_chunk_look_back=encoder_chunk_look_back,
                         decoder_chunk_look_back=decoder_chunk_look_back)
    for entry in res:
        transcribed_text.append(entry['text'])
final_transcript = ''.join(transcribed_text)
print(final_transcript)

rtf_avg: 0.211: 100%|██████████| 1/1 [00:00<00:00,  9.73it/s]                                                                                          

张晓静二十九岁她来上海五年了这天早晨他向往常一样站在卫生间的洗手池前一边快速的刷着牙一边浏览着手机上客户刚刚发来的消息他的脑子里飞快地计划着今天的工作你恐怕不会相信他是那种接到工作从不抱怨而且计划和学习能力极强从不怕吃苦的人毫不夸张的说任何老板遇到他都会对他的工作能力非常满意他就职职于海的一家设计公司司


# Manual Splitting method

In [ ]:
audio_path = r'C:\Users\Administrator\Desktop\Backend-LLM\audio_text_llm\SenseVoiceSmall\audio_sample\听故事学中文 Learn Chinese with 12 Stories - The Easiest Way to Improve Chinese.mp3'
audio_array, sampling_rate = librosa.load(audio_path, sr = None)
# List to hold transcriptions
transcriptions = []
model = AutoModel(
    model='paraformer-zh',
    # init_param = pretrained_model_path
    )
for start in range(0, total_samples, chunk_samples):
    end = min(start + chunk_samples, total_samples)
    chunk = audio_array[start:end]

    # Convert chunk to 16kHz if necessary
    # Note: Ensure your model accepts the sample rate of 16kHz or resample if needed
    chunk_resampled = librosa.resample(chunk, orig_sr = sampling_rate, target_sr = 16000)

    # Process the chunk
    res = model.generate(
        input=chunk_resampled,
        cache={},
        language="auto",
        use_itn=True,
        batch_size_s=60,
        merge_vad=True,
        merge_length_s=15,
    )

    # Get the transcribed text
    text = rich_transcription_postprocess(res[0]["text"])
    transcriptions.append(text)
    # Combine all transcriptions into one
full_transcription = " ".join(transcriptions)
# Save the transcription to a text file
with open("transcription_'paraformer-zh.txt", "w", encoding="utf-8") as file:
    file.write(full_transcription)


In [ ]:
# Resampling the Audio so it matches to the requirement of the model
audio_16KHz = librosa.resample(audio_array,
                               orig_sr=sampling_rate,
                
                               target_sr=16000)

# paraformer-zh model with vad_model


In [ ]:
model = AutoModel(
    model='paraformer-zh',
     vad_model="fsmn-vad",
     vad_kwargs={"max_single_segment_time": 30000},
    device="cuda:0",
    #punc_model = "ct-punc"
)

# en
res = model.generate(
    input=audio_array,
    cache={},
    language="zn",  # "zn", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)
# # Save the transcription to a text file
# with open("./outdir/transcription_paraformer-zh.txt", "w", encoding="utf-8") as file:
#     file.write(full_transcription)

# SenseVoiceSmall Model with vad model

In [ ]:
model = AutoModel(
    model='iic/SenseVoiceSmall',
     vad_model="fsmn-vad",
     vad_kwargs={"max_single_segment_time": 30000},
    device="cuda:0",
    #punc_model = "ct-punc"
)

# en
res = model.generate(
    input=audio_16KHz,
    cache={},
    language="zn",  # "zn", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)

In [75]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Load the processor and model
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import librosa
import numpy as np

# Set the sampling rate
sample_rate = 16000

def record_audio(duration=5):
    # Record audio for a specified duration
    audio = librosa.record(duration=duration, sr=sample_rate)
    return audio

# Example usage
audio_data = record_audio(duration=5)


In [ ]:
while True:
    audio_data = record_audio(duration=5)  # Record for 5 seconds

    # Convert audio data to the format required by the model
    audio_input = processor(audio_data, sampling_rate=sample_rate, return_tensors="pt")

    # Use the model to transcribe the audio
    with torch.no_grad():
        generated_ids = model.generate(**audio_input)

    # Decode the generated ids to get the transcription
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(transcription)
